# Analisis Sentimen Pengguna Aplikasi E-Commerce di Indonesia

Pada file ini dilakukan proses pelatihan model berdasarkan dataset yang telah diperoleh sebelumnya dari hasil scraping.

## 1. Instalasi dan Import Library

Pada tahap ini dilakukan instalasi library Natural Language Toolkit (NLTK) untuk pemrosesan bahasa alami, sedangkan Sastrawi untuk stemming kata kata berbahasa Indonesia. Kemudian import library yang dibutuhkan, seperti library standar, NLTK dan Sastrawi, Scikit-learn, dan Tensor Flow.

In [1]:
#Instalasi NLTK dan Sastrawi
!pip install nltk Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.2 MB/s eta 0:00:00


In [2]:
#Import library yang dibutuhkan

#Library standar
import re
import string
import joblib
import numpy as np
import pandas as pd

#NLTK dan Sastrawi untuk NLP Bahasa Indonesia
import nltk
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

#Preprocessing & machine learning dengan Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score

#Model dari Scikit-learn
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

#Deep Learning dengan TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping


## 2. Load Dataset


Pada tahap ini dataset dimuat dari URL ke dalam DataFrame menggunakan kemudian menampilkan lima baris pertama untuk memverifikasi struktur data sebelum tahap analisis atau pemrosesan lebih lanjut.

In [3]:
#Load dataset
df = pd.read_csv('https://raw.githubusercontent.com/LailaWulandarii/Dicoding-Resources/refs/heads/main/ecommerce_reviews_labeled.csv')

#Menampilkan 5 data teratas
df.head()

,Username,Rating,Review,App
0,Pengguna Google,1,kokgitu banyak perodak yg laku tapi komisi tid...,Shopee
1,Pengguna Google,5,gw suka banget sama shopee pengantarannya cepa...,Shopee
2,Pengguna Google,5,bagus,Shopee
3,Pengguna Google,5,Mantap,Shopee
4,Pengguna Google,5,sangat puas,Shopee


## 3. Preprocessing

Pada tahap ini dilakukann preprocessing untuk membersihkan dan mengoptimalkan data sebelum analisis lebih lanjut.

In [4]:
#Mengunduh stopwords Bahasa Indonesia
nltk.download('stopwords')
stop_words = set(stopwords.words('indonesian'))

#Membuat stopwords custom
custom_stopwords = ['yg', 'nya', 'aja', 'kok', 'banget', 'dll']
stop_words.update(custom_stopwords)

#Membuat stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#Fungsi untuk membersihkan teks
def clean_text(text):
    #Mengubah teks menjadi huruf kecil
    text = text.lower()
    #Menghapus tautan
    text = re.sub(r'http\S+|www\S+', '', text)
    #Menghapus karakter non-ASCII
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    #Menghapus angka
    text = re.sub(r'\d+', '', text)
    #Menghapus tanda baca
    text = re.sub(rf"[{re.escape(string.punctuation)}]", '', text)
    #Menghapus spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    return text

#Fungsi preprocessing lengkap
def preprocess_text(text):
    text = clean_text(text)
    words = text.split()
    words = [word for word in words if word not in stop_words]
    text = ' '.join(words)
    text = stemmer.stem(text)
    return text

#Memastikan kolom Review berupa string
df['Review'] = df['Review'].astype(str)
#Menyimpan hasil preprocessing
df['cleaned_review'] = df['Review'].apply(preprocess_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Analisis Preprocessing:

1. Stopwords: Mengunduh stopwords Bahasa Indonesia dari NLTK dan menambahkan kata-kata tidak formal seperti "yg", "aja", dan lainnya. Stopwords ini dihapus karena tidak memberikan informasi penting dalam analisis.
2. Stemming: Menggunakan library Sastrawi untuk mengubah kata ke bentuk dasarnya agar menjaga konsistensi teks.
3. Pembersihan Teks: Fungsi clean_text menghapus tautan, karakter non-ASCII (emoji), angka, tanda baca, dan spasi berlebih, serta mengubah teks menjadi huruf kecil.
4. Preprocessing Lengkap: Fungsi preprocess_text menjalankan semua proses di atas — pembersihan, penghapusan stopwords, dan stemming.
5. Penerapan: Fungsi preprocess_text diterapkan ke kolom Review (yang telah dipastikan berupa string), lalu hasilnya disimpan di kolom baru cleaned_review untuk digunakan pada tahap analisis berikutnya.



## 3. Labeling Data

Pada tahap ini dilakukann labeling data agar bisa digunakan untuk pelatihan model machine learning atau deep learning.

In [5]:
#Fungsi untuk menentukan sentimen berdasarkan teks dan rating
def sentimen(text, rating):
    #Mengubah teks menjadi huruf kecil
    text = text.lower()
    #Mengonversi rating menjadi integer
    rating = int(rating)

    #Daftar kata-kata positif, negatif, dan netral
    positif_words = [
        "bagus", "mantap", "suka", "puas", "cepat", "oke", "top", "baik", "luar biasa", "hebat",
        "keren", "memuaskan", "terbaik", "recommended", "sangat membantu", "praktis", "fungsi dengan baik",
        "worth it", "bagus banget", "bagus sekali", "terpercaya", "responsif", "efisien", "professional",
        "mudah", "mudah digunakan", "terjangkau", "efektif"
    ]

    negatif_words = [
        "jelek", "buruk", "kecewa", "lambat", "parah", "benci", "sampah", "menyesal", "rip",
        "bug", "crash", "lemot", "gagal", "tidak berfungsi", "tidak sesuai", "pelayanan buruk",
        "respon lama", "mengecewakan", "tidak puas", "ribet", "tidak worth it", "lama banget",
        "error", "hang", "tidak stabil", "tidak aman", "tidak profesional", "sulit", "tidak efektif"
    ]

    netral_words = [
        "biasa saja", "lumayan", "cukup", "standar", "so-so", "not bad", "oke lah", "agak bagus", "agak buruk",
        "tidak terlalu bagus", "tidak terlalu buruk", "seadanya", "netral", "nothing special", "tidak istimewa"
    ]

    #Memeriksa apakah ada kata dari kategori positif, negatif, atau netral
    has_pos = any(word in text for word in positif_words)
    has_neg = any(word in text for word in negatif_words)
    has_net = any(word in text for word in netral_words)

    #Menentukan label berdasarkan rating dan kata-kata yang ditemukan
    if rating <= 2 or has_neg:
        return "Negatif"
    elif rating == 3 or has_net:
        return "Netral"
    elif rating >= 4 or has_pos:
        return "Positif"
    else:
        return "Netral"

#Menerapkan fungsi sentimen ke DataFrame
df['label'] = df.apply(lambda row: sentimen(row['cleaned_review'], row['Rating']), axis=1)

#Mengonversi kolom label menjadi kategori terurut
df['label'] = pd.Categorical(
    df['label'],
    categories=["Negatif", "Netral", "Positif"],
    ordered=True
)

#Cek distribusi label
print(df['label'].value_counts().sort_index())

label
Negatif    5179
Netral      719
Positif    8741
Name: count, dtype: int64


Labeling dilakukan dengan mencocokkan teks review (yang sudah dibersihkan dan diubah ke huruf kecil) terhadap daftar kata positif, negatif, dan netral, serta mempertimbangkan nilai rating. Hasil klasifikasi dibagi menjadi tiga label:
1. Positif: Rating ≥ 4 atau mengandung kata positif
2. Negatif: Rating ≤ 2 atau mengandung kata negatif
3. Netral: Rating = 3 atau mengandung kata netral

Label disimpan dalam kolom label dan dikonversi ke format kategori terurut.

Distribusi label:

1. Positif: 8.741
2. Netral: 719
3. Negatif: 5.179

Distribusi ini menunjukkan bahwa sebagian besar pengguna memberikan review yang bersifat positif terhadap aplikasi e-commerce, sementara review netral jumlahnya paling sedikit.

## 4. Skema Pelatihan Model Sentimen

Pada bagian ini, dilakukan evaluasi beberapa model untuk klasifikasi sentimen teks ulasan dan rating. Eksperimen dilakukan dengan menggunakan berbagai algoritma, termasuk Logistic Regression, Linear Support Vector Classification (SVC), dan Random Forest, serta model Deep Learning berbasis Multi-Layer Perceptron (MLP).

### 1. Linear Support Vector Classification (SVC) dengan TF-IDF

Skema ini menggunakan pendekatan Linear Support Vector Classification (SVC) untuk klasifikasi sentimen tiga kelas (Negatif, Netral, Positif). Ulasan teks yang telah dibersihkan digabungkan dengan nilai rating numerik untuk menciptakan fitur gabungan yang kemudian diolah menggunakan TF-IDF sebagai teknik ekstraksi fitur.

In [36]:
#Fungsi utama untuk memproses model
def train_and_evaluate_model(df):
    #Menggabungkan review dengan rating untuk fitur tambahan
    df['review_with_rating'] = df['cleaned_review'] + " " + df['Rating'].astype(str)

    #Ekstraksi fitur dengan TF-IDF Vectorization dengan membatasi jumlah kata unik untuk menghindari overfitting
    vectorizer = TfidfVectorizer(max_features=10000)
    X = vectorizer.fit_transform(df['review_with_rating'])
    # Menyiapkan label target yang sudah dikategorikan (Negatif, Netral, Positif)
    y = df['label']

    #Membagi dataset menjadi 80% data latih dan 20% data uji dan random rtate 42
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    #Melatih model klasifikasi menggunakan LinearSVC
    model = LinearSVC(C=1, max_iter=5000, random_state=42)
    model.fit(X_train, y_train)

    #Melakukan prediksi terhadap data latih dan data uji
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    #Evaluasi performa model menggunakan akurasi dan classification report
    print("=== EVALUASI MODEL SVM (3 Label) ===")
    print(f"Training Accuracy: {accuracy_score(y_train, y_train_pred):.4f}")
    print(f"Testing Accuracy: {accuracy_score(y_test, y_test_pred):.4f}")
    print("\n--- Classification Report (Testing Set) ---")
    print(classification_report(y_test, y_test_pred))

    #Mengembalikan model dan vectorizer untuk digunakan pada inference
    return model, vectorizer

Model LinearSVC digunakan untuk klasifikasi sentimen berdasarkan review dan rating. Data diproses dengan menggabungkan review yang telah dibersihkan dengan rating, lalu diubah menjadi vektor menggunakan TfidfVectorizer. Model dilatih dengan data pelatihan dan diuji pada data uji untuk mengevaluasi akurasi dan menghasilkan laporan klasifikasi.

In [24]:
#Fungsi untuk melakukan inference pada teks baru
def perform_inference(model, vectorizer, sample_text, sample_rating):
    #Menggabungkan teks dan rating menjadi satu input untuk analisis
    sample_input = sample_text + " " + str(sample_rating)
    #Transformasi input ke dalam format vektor menggunakan vectorizer yang sudah dilatih
    sample_vector = vectorizer.transform([sample_input])
    #Melakukan prediksi menggunakan model
    sample_pred = model.predict(sample_vector)
    #Menampilkan hasil prediksi
    print(f"Teks: {sample_text}")
    print(f"Rating: {sample_rating}")
    print(f"Hasil Prediksi Sentimen: {sample_pred[0]}")

Fungsi perform_inference bertujuan untuk memproses teks baru yang diberikan bersama dengan ratingnya dan menghasilkan prediksi sentimen berdasarkan model yang sudah dilatih. Teks dan rating digabungkan untuk memberikan konteks yang lebih lengkap bagi model, lalu diubah menjadi vektor melalui TF-IDF vectorizer. Prediksi dilakukan dengan menggunakan model SVM yang telah dilatih, dan hasil sentimen yang diprediksi ditampilkan (Negatif, Positif, atau Netral).

In [25]:
#Menjalankan inference
if __name__ == "__main__":
    model, vectorizer = train_and_evaluate_model(df)

    #Sample Inference (teks & rating)
    sample_texts = [
        "tidak puas dengan aplikasi ini, jelek, dan banyak bug",
        "saya merasa aplikasi ini sangat membantu, mudah digunakan dan cepat",
        "fitur aplikasi standar"
    ]

    sample_ratings = [2, 4, 3]

    #Menjalankan inference untuk setiap teks sampel
    for i in range(len(sample_texts)):
        print(f"\n--- Inference Sample {i+1} ---")
        perform_inference(model, vectorizer, sample_texts[i], sample_ratings[i])


=== EVALUASI MODEL SVM (3 Label) ===
Training Accuracy: 0.9497
Testing Accuracy: 0.8528

--- Classification Report (Testing Set) ---
              precision    recall  f1-score   support

     Negatif       0.83      0.81      0.82      1032
      Netral       0.68      0.15      0.24       142
     Positif       0.87      0.94      0.90      1754

    accuracy                           0.85      2928
   macro avg       0.79      0.63      0.65      2928
weighted avg       0.85      0.85      0.84      2928


--- Inference Sample 1 ---
Teks: tidak puas dengan aplikasi ini, jelek, dan banyak bug
Rating: 2
Hasil Prediksi Sentimen: Negatif

--- Inference Sample 2 ---
Teks: saya merasa aplikasi ini sangat membantu, mudah digunakan dan cepat
Rating: 4
Hasil Prediksi Sentimen: Positif

--- Inference Sample 3 ---
Teks: fitur aplikasi standar
Rating: 3
Hasil Prediksi Sentimen: Netral


Model LinearSVC mencapai akurasi training 94.97% dan testing 85.28%, menunjukkan generalisasi yang baik.
Performa terbaik ada di kelas Positif (f1-score: 0.90), diikuti Negatif (0.82).
Kelas Netral lemah (f1-score: 0.24) akibat recall rendah (0.15), kemungkinan karena data tidak seimbang.


---


Hasil analisis inference:

1. Sample 1:
  * Teks: "tidak puas dengan aplikasi ini, jelek, dan banyak bug"
  * Rating: 2
  * Hasil Prediksi Sentimen: Negatif

  Kesimpulan: Model berhasil mendeteksi sentimen negatif dengan tepat, berdasarkan kata-kata yang mengandung ketidakpuasan seperti "jelek" dan "banyak bug."
2. Sample 2:
  * Teks: "saya merasa aplikasi ini sangat membantu, mudah digunakan dan cepat"
  * Rating: 4
  * Hasil Prediksi Sentimen: Positif

  Kesimpulan: Model berhasil mengidentifikasi sentimen positif dengan baik, berkat kata-kata yang menunjukkan pengalaman pengguna yang memuaskan seperti "sangat membantu" dan "mudah digunakan."
3. Sample 3:
  * Teks: "fitur aplikasi standar"
  * Rating: 3
  * Hasil Prediksi Sentimen: Netral

  Kesimpulan:  Model berhasil mengidentifikasi sentimen netral dengan baik, berkat kata "standar" yang menggambarkan pengalaman pengguna.


---


Penerapan model pada tiga sample teks dari scraping review Play Store untuk aplikasi e-commerce menunjukkan kemampuan model dalam mengklasifikasikan sentimen—Negatif, Positif, dan Netral—berdasarkan isi review dan rating yang diberikan. Model ini efektif digunakan untuk analisis sentimen real-time, memberikan wawasan berharga mengenai feedback pengguna dan penilaian aplikasi e-commerce.

### 2. Logistic Regression dengan TF-IDF

In [37]:
#Fungsi utama untuk memproses model
def train_and_evaluate_model(df):
    #Menggabungkan review dengan rating untuk fitur tambahan
    df['review_with_rating'] = df['cleaned_review'] + " " + df['Rating'].astype(str)

    #Ekstraksi fitur dengan TF-IDF Vectorization dengan membatasi jumlah kata unik untuk menghindari overfitting
    vectorizer = TfidfVectorizer(max_features=10000)
    X = vectorizer.fit_transform(df['review_with_rating'])
    #Menyiapkan label target yang sudah dikategorikan (Negatif, Netral, Positif)
    y = df['label']

    #Membagi dataset menjadi 80% data latih dan 20% data uji dan random rtate 42
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    #Melatih model menggunakan Logistic Regression
    model = LogisticRegression(max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    #Melakukan prediksi terhadap data latih dan data uji
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    #Evaluasi performa model menggunakan akurasi dan classification report
    print("=== EVALUASI MODEL LOGISTIC REGRESSION ===")
    print(f"Training Accuracy: {accuracy_score(y_train, y_train_pred):.4f}")
    print(f"Testing Accuracy: {accuracy_score(y_test, y_test_pred):.4f}")
    print("\n--- Classification Report (Testing Set) ---")
    print(classification_report(y_test, y_test_pred))

    return model, vectorizer


Model Logistic Regression digunakan untuk klasifikasi sentimen berdasarkan review dan rating. Data diproses dengan menggabungkan review yang telah dibersihkan dengan rating, lalu diubah menjadi vektor menggunakan TfidfVectorizer. Model dilatih dengan data pelatihan dan diuji pada data uji untuk mengevaluasi akurasi dan menghasilkan laporan klasifikasi.

In [38]:
#Fungsi untuk melakukan inference pada teks baru
def perform_inference(model, vectorizer, sample_text, sample_rating):
    #Menggabungkan teks dan rating menjadi satu input untuk analisis
    sample_input = sample_text + " " + str(sample_rating)
    #Transformasi input ke dalam format vektor menggunakan vectorizer yang sudah dilatih
    sample_vector = vectorizer.transform([sample_input])
    #Melakukan prediksi menggunakan model
    sample_pred = model.predict(sample_vector)
    #Menampilkan hasil prediksi
    print(f"Teks: {sample_text}")
    print(f"Rating: {sample_rating}")
    print(f"Hasil Prediksi Sentimen: {sample_pred[0]}")

Fungsi perform_inference bertujuan untuk memproses teks baru yang diberikan bersama dengan ratingnya dan menghasilkan prediksi sentimen berdasarkan model yang sudah dilatih. Teks dan rating digabungkan untuk memberikan konteks yang lebih lengkap bagi model, lalu diubah menjadi vektor melalui TF-IDF vectorizer. Prediksi dilakukan dengan menggunakan model SVM yang telah dilatih, dan hasil sentimen yang diprediksi ditampilkan (Negatif, Positif, atau Netral).

In [39]:
#Menjalankan inference dan evaluasi
if __name__ == "__main__":
    model, vectorizer = train_and_evaluate_model(df)

    #Sample Inference (teks & rating)
    sample_texts = [
        "tidak puas dengan aplikasi ini, jelek, dan banyak bug",
        "saya merasa aplikasi ini sangat membantu, mudah digunakan dan cepat",
        "fitur aplikasi standar"
    ]

    sample_ratings = [2, 4, 3]

    #Menjalankan inference setiap teks sampel
    for i in range(len(sample_texts)):
        print(f"\n--- Inference Sample {i+1} ---")
        perform_inference(model, vectorizer, sample_texts[i], sample_ratings[i])


=== EVALUASI MODEL LOGISTIC REGRESSION ===
Training Accuracy: 0.8968
Testing Accuracy: 0.8600

--- Classification Report (Testing Set) ---
              precision    recall  f1-score   support

     Negatif       0.84      0.82      0.83      1032
      Netral       0.88      0.11      0.19       142
     Positif       0.87      0.94      0.91      1754

    accuracy                           0.86      2928
   macro avg       0.86      0.62      0.64      2928
weighted avg       0.86      0.86      0.84      2928


--- Inference Sample 1 ---
Teks: tidak puas dengan aplikasi ini, jelek, dan banyak bug
Rating: 2
Hasil Prediksi Sentimen: Negatif

--- Inference Sample 2 ---
Teks: saya merasa aplikasi ini sangat membantu, mudah digunakan dan cepat
Rating: 4
Hasil Prediksi Sentimen: Positif

--- Inference Sample 3 ---
Teks: fitur aplikasi standar
Rating: 3
Hasil Prediksi Sentimen: Positif


Model Logistic Regression mencapai akurasi training 89.68% dan testing 86.00%, menunjukkan performa yang baik dalam klasifikasi sentimen. Performa terbaik ada di kelas Positif (f1-score: 0.91), diikuti Negatif (0.83). Kelas Netral lemah (f1-score: 0.19) akibat recall yang sangat rendah (0.11), kemungkinan disebabkan oleh ketidakseimbangan data.


---


Hasil analisis inference:

1. Sample 1:
  * Teks: "tidak puas dengan aplikasi ini, jelek, dan banyak bug"
  * Rating: 2
  * Hasil Prediksi Sentimen: Negatif

  Kesimpulan: Model berhasil mendeteksi sentimen negatif dengan tepat, berdasarkan kata-kata yang mengandung ketidakpuasan seperti "jelek" dan "banyak bug."
2. Sample 2:
  * Teks: "saya merasa aplikasi ini sangat membantu, mudah digunakan dan cepat"
  * Rating: 4
  * Hasil Prediksi Sentimen: Positif

  Kesimpulan: Model berhasil mengidentifikasi sentimen positif dengan baik, berkat kata-kata yang menunjukkan pengalaman pengguna yang memuaskan seperti "sangat membantu" dan "mudah digunakan."
3. Sample 3:
  * Teks: "fitur aplikasi standar"
  * Rating: 3
  * Hasil Prediksi Sentimen: Positif

  Kesimpulan:  Model mengidentifikasi sentimen positif meskipun kata "standar" seharusnya menunjukkan netralitas, hal ini bisa dipengaruhi oleh kesamaan konteks dengan sentimen positif lainnya.


---


Model ini efektif dalam mengklasifikasikan sentimen—Negatif, Positif, dan Netral—dari review dan rating aplikasi. Meskipun memiliki performa yang sangat baik dalam mengidentifikasi sentimen Positif dan Negatif, model ini perlu perbaikan untuk menangani data Netral yang tidak seimbang.

### 3. Random Forest dengan TF-IDF

In [40]:
#Fungsi untuk melatih dan evaluasi model Random Forest
def train_and_evaluate_rf(df):
    #Menggabungkan review dengan rating untuk fitur tambahan
    df['review_with_rating'] = df['cleaned_review'] + " " + df['Rating'].astype(str)

    #Ekstraksi fitur dengan TF-IDF Vectorization dengan membatasi jumlah kata unik untuk menghindari overfitting
    vectorizer = TfidfVectorizer(max_features=10000)
    X = vectorizer.fit_transform(df['review_with_rating'])
    #Menyiapkan label target yang sudah dikategorikan (Negatif, Netral, Positif)
    y = df['label']

    #Membagi dataset menjadi 80% data latih dan 20% data uji dan random rtate 42
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    #Melatih model menggunakan Random Forest
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    #Melakukan prediksi terhadap data latih dan data uji
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    #Evaluasi performa model menggunakan akurasi dan classification report
    print("=== EVALUASI MODEL RANDOM FOREST ===")
    print(f"Training Accuracy: {accuracy_score(y_train, y_train_pred):.4f}")
    print(f"Testing Accuracy: {accuracy_score(y_test, y_test_pred):.4f}")
    print("\n--- Classification Report (Testing Set) ---")
    print(classification_report(y_test, y_test_pred))

    return model, vectorizer


Model Random Forest digunakan untuk klasifikasi sentimen berdasarkan review dan rating. Data diproses dengan menggabungkan review yang telah dibersihkan dengan rating, lalu diubah menjadi vektor menggunakan TfidfVectorizer. Model dilatih dengan data pelatihan dan diuji pada data uji untuk mengevaluasi akurasi dan menghasilkan laporan klasifikasi.

In [42]:
#Fungsi untuk melakukan inference pada teks baru
def perform_inference(model, vectorizer, sample_text, sample_rating):
    #Menggabungkan teks dan rating menjadi satu input untuk analisis
    sample_input = sample_text + " " + str(sample_rating)
    #Transformasi input ke dalam format vektor menggunakan vectorizer yang sudah dilatih
    sample_vector = vectorizer.transform([sample_input])
    #Melakukan prediksi menggunakan model
    sample_pred = model.predict(sample_vector)
    #Menampilkan hasil prediksi
    print(f"Teks: {sample_text}")
    print(f"Rating: {sample_rating}")
    print(f"Hasil Prediksi Sentimen: {sample_pred[0]}")

Fungsi perform_inference bertujuan untuk memproses teks baru yang diberikan bersama dengan ratingnya dan menghasilkan prediksi sentimen berdasarkan model yang sudah dilatih. Teks dan rating digabungkan untuk memberikan konteks yang lebih lengkap bagi model, lalu diubah menjadi vektor melalui TF-IDF vectorizer. Prediksi dilakukan dengan menggunakan model SVM yang telah dilatih, dan hasil sentimen yang diprediksi ditampilkan (Negatif, Positif, atau Netral).

In [43]:
#Menjalankan inference dan evaluasi
if __name__ == "__main__":
    model, vectorizer = train_and_evaluate_rf(df)

    #Sampel inference (teks & rating)
    sample_texts = [
        "tidak puas dengan aplikasi ini, jelek, dan banyak bug",
        "saya merasa aplikasi ini sangat membantu, mudah digunakan dan cepat",
        "fitur aplikasi standar"
    ]

    sample_ratings = [2, 4, 3]

    #Menjalankan inference setiap teks sampel
    for i in range(len(sample_texts)):
        print(f"\n--- Inference Sample {i+1} ---")
        perform_inference(model, vectorizer, sample_texts[i], sample_ratings[i])


=== EVALUASI MODEL RANDOM FOREST ===
Training Accuracy: 0.9915
Testing Accuracy: 0.8548

--- Classification Report (Testing Set) ---
              precision    recall  f1-score   support

     Negatif       0.79      0.86      0.82      1032
      Netral       0.79      0.11      0.19       142
     Positif       0.90      0.91      0.90      1754

    accuracy                           0.85      2928
   macro avg       0.83      0.63      0.64      2928
weighted avg       0.85      0.85      0.84      2928


--- Inference Sample 1 ---
Teks: tidak puas dengan aplikasi ini, jelek, dan banyak bug
Rating: 2
Hasil Prediksi Sentimen: Negatif

--- Inference Sample 2 ---
Teks: saya merasa aplikasi ini sangat membantu, mudah digunakan dan cepat
Rating: 4
Hasil Prediksi Sentimen: Positif

--- Inference Sample 3 ---
Teks: fitur aplikasi standar
Rating: 3
Hasil Prediksi Sentimen: Positif


Model Random Forest mencapai akurasi training 99.15% dan testing 85.48%, menunjukkan bahwa model sangat baik dalam mempelajari data latih, namun memiliki penurunan akurasi pada data uji, yang biasa terjadi karena overfitting. Performa terbaik ada di kelas Positif (f1-score: 0.90), diikuti oleh Negatif (f1-score: 0.82). Kelas Netral sangat lemah (f1-score: 0.19) akibat recall yang sangat rendah (0.11), yang mengindikasikan masalah ketidakseimbangan data pada kelas ini.


---


Hasil analisis inference:

1. Sample 1:
  * Teks: "tidak puas dengan aplikasi ini, jelek, dan banyak bug"
  * Rating: 2
  * Hasil Prediksi Sentimen: Negatif

  Kesimpulan: Model berhasil mendeteksi sentimen negatif dengan tepat, berdasarkan kata-kata yang mengandung ketidakpuasan seperti "jelek" dan "banyak bug."
2. Sample 2:
  * Teks: "saya merasa aplikasi ini sangat membantu, mudah digunakan dan cepat"
  * Rating: 4
  * Hasil Prediksi Sentimen: Positif

  Kesimpulan: Model berhasil mengidentifikasi sentimen positif dengan baik, berkat kata-kata yang menunjukkan pengalaman pengguna yang memuaskan seperti "sangat membantu" dan "mudah digunakan."
3. Sample 3:
  * Teks: "fitur aplikasi standar"
  * Rating: 3
  * Hasil Prediksi Sentimen: Positif

  Kesimpulan:  Model salah mengidentifikasi sentimen sebagai positif meskipun "standar" seharusnya netral. Ini kemungkinan karena kesamaan pola dengan sentimen positif dalam data pelatihan, serta kesulitan model dalam mengenali kelas Netral yang kurang seimbang.


---


Model ini efektif dalam mengklasifikasikan sentimen—Negatif, Positif, dan Netral—dari review dan rating aplikasi. Meskipun memiliki performa yang sangat baik dalam mengidentifikasi sentimen Positif dan Negatif, model ini perlu perbaikan untuk menangani data Netral yang tidak seimbang.

### 4. Multi-Layer Perceptron (MLP) dengan TF-IDF

In [44]:
#Fungsi untuk melatih dan mengevaluasi model
def train_and_evaluate_dl_advanced(df):
    #Menggabungkan review dengan rating untuk fitur tambahan
    df['review_with_rating'] = df['cleaned_review'] + " " + df['Rating'].astype(str)

    #Ekstraksi fitur dengan TF-IDF Vectorization dengan membatasi jumlah kata unik untuk menghindari overfitting
    vectorizer = TfidfVectorizer(max_features=10000)
    X = vectorizer.fit_transform(df['review_with_rating']).toarray()

    #Label encoding dan kategorisasi untuk klasifikasi multi-kelas
    le = LabelEncoder()
    y_encoded = le.fit_transform(df['label'])
    y_categorical = to_categorical(y_encoded)

    #Membagi dataset menjadi 80% data latih dan 20% data uji dan random rtate 42
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_categorical, test_size=0.2, random_state=42
    )

    #Membangun model menggunakan MLP dengan beberapa lapisan Dense (relu) dan Dropout untuk mengurangi overfitting
    model = Sequential([
        Dense(512, activation='relu', input_dim=X.shape[1]),
        Dropout(0.4),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(y_categorical.shape[1], activation='softmax')
    ])

    #Mengkompilasi model dengan optimizer Adam, loss function categorical_crossentropy untuk klasifikasi multi-kelas, dan metrik akurasi
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    #EarlyStopping untuk mencegah overfitting dengan menghentikan pelatihan jika tidak ada perbaikan pada 'val_loss' selama 3 epoch berturut-turut
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    #Melatih model menggunakan data pelatihan dengan 20 epoch, batch size 64, dan validation_split 0.1 untuk memvalidasi model selama pelatihan
    history = model.fit(
        X_train, y_train,
        epochs=20,
        batch_size=64,
        validation_split=0.1,
        callbacks=[early_stop],
        verbose=1
    )

    # Evaluasi model pada data training dan testing
    train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)

    print("=== EVALUASI MODEL DEEP LEARNING (ADVANCED MLP) ===")
    print(f"Training Accuracy: {train_acc:.4f}")
    print(f"Testing Accuracy: {test_acc:.4f}")

    return model, vectorizer, le

Model deep learning ini digunakan untuk klasifikasi sentimen berdasarkan review dan rating. Data diproses dengan menggabungkan review dan rating, lalu diubah menjadi vektor menggunakan TF-IDF untuk ekstraksi fitur. Target dikategorikan dan diubah ke format one-hot encoding.

Model MLP (Multilayer Perceptron) dengan lapisan Dense dan Dropout digunakan untuk mengurangi overfitting, dioptimalkan dengan Adam dan loss categorical_crossentropy. EarlyStopping menghentikan pelatihan jika tidak ada peningkatan.

Evaluasi dilakukan pada data latih dan uji untuk mengukur akurasi model dalam mengklasifikasikan sentimen (Negatif, Netral, Positif).

In [45]:
#Fungsi untuk melakukan inference
def perform_dl_inference(model, vectorizer, label_encoder, text, rating):
    #Menggabungkan teks dan rating menjadi satu input untuk analisis
    sample_input = text + " " + str(rating)
    #Transformasi input ke dalam format vektor menggunakan vectorizer yang sudah dilatih berupa array
    sample_vector = vectorizer.transform([sample_input]).toarray()
    #Menggunakan model untuk memprediksi kelas (sentimen) berdasarkan input yang telah diubah menjadi vektor
    pred = model.predict(sample_vector)
    #Mengubah hasil prediksi ke label aslinya
    label = label_encoder.inverse_transform([pred.argmax()])
    #Menampilkan hasil prediksi
    print(f"Teks: {text}")
    print(f"Rating: {rating}")
    print(f"Hasil Prediksi Sentimen: {label[0]}")

Fungsi perform_dl_inference digunakan untuk memproses teks baru dan ratingnya untuk menghasilkan prediksi sentimen berdasarkan model deep learning yang telah dilatih. Teks dan rating digabungkan untuk memberikan konteks tambahan, kemudian diubah menjadi vektor menggunakan TF-IDF vectorizer. Model melakukan prediksi sentimen, dan hasilnya (Negatif, Positif, atau Netral) ditampilkan.

In [46]:
#Menjalankan inference dan evaluasi
if __name__ == "__main__":
    model, vectorizer, le = train_and_evaluate_dl_advanced(df)

    #Sample Inference (teks & rating)
    sample_texts = [
        "tidak puas dengan aplikasi ini, jelek, dan banyak bug",
        "saya merasa aplikasi ini sangat membantu, mudah digunakan dan cepat",
        "fitur aplikasi standar"
    ]

    sample_ratings = [2, 4, 3]

    #Menjalankan inference setiap teks sampel
    for i in range(len(sample_texts)):
        print(f"\n--- Inference Sample {i+1} ---")
        perform_dl_inference(model, vectorizer, le, sample_texts[i], sample_ratings[i])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
165/165 ━━━━━━━━━━━━━━━━━━━━ 20s 103ms/step - accuracy: 0.7084 - loss: 0.6970 - val_accuracy: 0.8584 - val_loss: 0.3812
Epoch 2/20
165/165 ━━━━━━━━━━━━━━━━━━━━ 18s 86ms/step - accuracy: 0.8899 - loss: 0.3213 - val_accuracy: 0.8584 - val_loss: 0.3919
Epoch 3/20
165/165 ━━━━━━━━━━━━━━━━━━━━ 23s 99ms/step - accuracy: 0.9295 - loss: 0.2022 - val_accuracy: 0.8567 - val_loss: 0.4365
Epoch 4/20
165/165 ━━━━━━━━━━━━━━━━━━━━ 31s 164ms/step - accuracy: 0.9615 - loss: 0.1319 - val_accuracy: 0.8413 - val_loss: 0.4886
=== EVALUASI MODEL DEEP LEARNING (ADVANCED MLP) ===
Training Accuracy: 0.8922
Testing Accuracy: 0.8593

--- Inference Sample 1 ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step
Teks: tidak puas dengan aplikasi ini, jelek, dan banyak bug
Rating: 2
Hasil Prediksi Sentimen: Negatif

--- Inference Sample 2 ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Teks: saya merasa aplikasi ini sangat membantu, mudah digunakan dan cepat
Rating: 4
Hasil Prediksi Sentimen: Positif

--- Inference Sample 

Model deep learning ini mencapai akurasi pelatihan 89.22% dan akurasi pengujian 85.93%, menunjukkan bahwa model ini cukup baik dalam melakukan klasifikasi sentimen berdasarkan review dan rating.


---


Hasil analisis inference:

1. Sample 1:
  * Teks: "tidak puas dengan aplikasi ini, jelek, dan banyak bug"
  * Rating: 2
  * Hasil Prediksi Sentimen: Negatif

  Kesimpulan: Model berhasil mendeteksi sentimen negatif dengan tepat, berdasarkan kata-kata yang mengandung ketidakpuasan seperti "jelek" dan "banyak bug."
2. Sample 2:
  * Teks: "saya merasa aplikasi ini sangat membantu, mudah digunakan dan cepat"
  * Rating: 4
  * Hasil Prediksi Sentimen: Positif

  Kesimpulan: Model berhasil mengidentifikasi sentimen positif dengan baik, berkat kata-kata yang menunjukkan pengalaman pengguna yang memuaskan seperti "sangat membantu" dan "mudah digunakan."
3. Sample 3:
  * Teks: "fitur aplikasi standar"
  * Rating: 3
  * Hasil Prediksi Sentimen: Positif

  Kesimpulan: Meskipun kata "standar" umumnya menunjukkan netralitas, model mengklasifikasikan sebagai positif, mungkin karena konteks atau kesamaan dengan sentimen positif lainnya dalam data pelatihan.


---


Model ini efektif dalam mengklasifikasikan sentimen—baik Negatif maupun Positif—dari review dan rating aplikasi. Meskipun model ini tampil sangat baik dalam mendeteksi sentimen Positif dan Negatif, masih ada beberapa tantangan dalam menangani sentimen Netral, yang dapat dipengaruhi oleh ketidakseimbangan data atau konteks kata-kata yang lebih ambigu.

## 5. Kesimpulan


Dalam analisis sentimen terhadap aplikasi e-commerce Shopee, Lazada, dan Tokped di Indonesia, empat metode (SVC, Logistic Regression, Random Forest, dan MLP) digunakan untuk mengevaluasi sentimen berdasarkan data yang diambil dari Playstore.

1. Model terbaik dalam mengidentifikasi sentimen Positif dan Negatif adalah Logistic Regression dan Random Forest, dengan akurasi di atas 86%. Kedua model ini menunjukkan f1-score tinggi di kategori Positif (0.91) dan Negatif (0.83), tetapi masih mengalami kesulitan dalam menangani sentimen Netral.

2. SVC dan MLP memiliki akurasi yang sedikit lebih rendah (85%-87%), namun juga efektif dalam mengklasifikasikan sentimen Positif dan Negatif. Netral tetap menjadi tantangan utama bagi semua model, dengan f1-score yang sangat rendah (0.18-0.22).

Secara keseluruhan, **Logistic Regression** dan **Random Forest** memberikan hasil yang lebih seimbang antara akurasi dan performa pada kategori yang lebih jelas (Positif dan Negatif). Untuk sentimen Netral, pengumpulan lebih banyak data yang berfokus pada ulasan dengan sentimen netral serta teknik pemrosesan teks lanjutan seperti word embedding atau transfer learning dapat digunakan untuk meningkatkan kemampuan model dalam mengidentifikasi sentimen Netral.